# Library

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# fold path

In [3]:
drive = 'E'

In [4]:
raw_fold = f'{drive}:/data/big2/BD1/raw'
ods_fold = f'{drive}:/data/big2/ODS'
sample_fold = f'{drive}:/data/big2/BD1/sample'
analysis_fold = f'{drive}:/data/big2/BD1/an'

# Load

## 병합 result 최종 파일

In [5]:
# about 15.1s
name = '4등급 차량 리스트 양식 result 최종 Grade 추가(2023.03.31_PM.07.32.15)'
file_name = f'{name}.csv'
df = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False, encoding='cp949')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1497675 entries, 0 to 1497674
Data columns (total 32 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   차대번호      1497675 non-null  object 
 1   차량번호      1497675 non-null  object 
 2   제원관리번호    1497675 non-null  object 
 3   차종        1497675 non-null  object 
 4   용도        1497675 non-null  object 
 5   최초등록일     1497675 non-null  int64  
 6   연식        1497675 non-null  int64  
 7   제작일자      1330654 non-null  float64
 8   검사유효일     1497663 non-null  float64
 9   배출가스인증번호  1377719 non-null  object 
 10  등급        1497675 non-null  int64  
 11  차명        1497675 non-null  object 
 12  차종분류      1497675 non-null  object 
 13  차종유형      1497675 non-null  object 
 14  자동차형식     1497399 non-null  object 
 15  제작사명      1488035 non-null  object 
 16  연료        1497675 non-null  object 
 17  엔진형식      1495843 non-null  object 
 18  총중량       1497674 non-null  float64
 19  적재중량      1497333 non

# 전처리

In [6]:
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', 'Grade'],
      dtype='object')

## 4등급 차량만

In [7]:
df['등급_수정'].value_counts(dropna=False)

4.00    1149290
NaN      348385
Name: 등급_수정, dtype: int64

In [10]:
df1 = df.loc[df['등급_수정'] == 4].reset_index(drop=True)
df1.shape

(1149290, 32)

## 중복 차대번호 확인
- 최종등록일자, 배출가스검사유효일자 최신 데이터 사용

In [11]:
df1.shape[0], len(df1['차대번호'].unique())

(1149290, 1149290)

## 연료 확인(경유)

In [12]:
df1['연료'].value_counts(dropna=False)

경유    1149290
Name: 연료, dtype: int64

## 차종 확인

In [13]:
df1['차종'].value_counts(dropna=False)

승용    525013
화물    485307
승합    132524
특수      6446
Name: 차종, dtype: int64

## 차종유형 확인

In [14]:
df1['차종유형'].value_counts()

중형    531633
소형    443250
대형    174406
경형         1
Name: 차종유형, dtype: int64

## 연식 확인

In [15]:
df1['연식'].describe()

count   1149290.00
mean       2009.43
std           1.52
min        1997.00
25%        2008.00
50%        2010.00
75%        2011.00
max        2018.00
Name: 연식, dtype: float64

### 차령 정보 계산 및 데이터 추가

In [16]:
current_yr = int(datetime.today().strftime("%Y"))
current_yr

2023

In [17]:
df1['차령'] = current_yr - df1['연식']
df1['차령'].describe()

count   1149290.00
mean         13.57
std           1.52
min           5.00
25%          12.00
50%          13.00
75%          15.00
max          26.00
Name: 차령, dtype: float64

## 시도명, 시군구명 수정
- 시군구명 앞지역명만 추출
- 세종틀별자치시 -> 세종시

In [18]:
df1[['시도명', '시군구명', '차종']].isnull().sum()

시도명     0
시군구명    0
차종      0
dtype: int64

In [19]:
df1['시도명'].unique()

array(['대전광역시', '부산광역시', '경기도', '제주특별자치도', '경상남도', '강원도', '전라남도', '인천광역시',
       '서울특별시', '충청북도', '충청남도', '대구광역시', '전라북도', '울산광역시', '경상북도', '광주광역시',
       '세종특별자치시'], dtype=object)

In [20]:
df1[['시도명', '시군구명']].tail()

,시도명,시군구명
1149285,충청남도,천안시 동남구
1149286,전라북도,김제시
1149287,전라북도,완주군
1149288,경기도,시흥시
1149289,충청남도,태안군


In [21]:
df1['시군구명_수정'] = df1['시군구명'].str.split(' ').str[0]

In [22]:
df1['시군구명_수정'].tail()

1149285    천안시
1149286    김제시
1149287    완주군
1149288    시흥시
1149289    태안군
Name: 시군구명_수정, dtype: object

In [23]:
df1.loc[df1['시군구명_수정'] == '세종특별자치시', '시군구명_수정'].shape

(6773,)

In [24]:
df1.loc[df1['시군구명_수정'] == '세종특별자치시', '시군구명_수정'] = '세종시'

In [25]:
df1.loc[df1['시군구명_수정'] == '세종특별자치시', '시군구명_수정'].shape

(0,)

# 자동차-엔진 가열(Hot-start) 배출

## 배출량 산정
$E_{i, j} = VKT \times {EF_i \over {1000}} \times DF \times (1 - {R \over {100}})$

$E_{i, j}$ : 자동차 $j$의 도로주행시 발생한 오염물질 $i$의 배출량($kg/yr$)

$VKT$ : 주행거리($km/yr$)

$EF_i$ : 차종별, 연료별, 연식별, 차속별 배출계수($g/km$)

$DF$ : 열화계수

$R$ : 저감장치 부착 효율(%)

### 주행거리(VKT)
- "용도별_차종별_시군구별_자동차주행거리" 자료(KOSIS)의 1일 평균주행거리 * 365 = 주행거리(km/yr)

#### [로드] 1일평균주행거리(2021)

In [26]:
# 1일 평균 주행거리 자료 로드
run = pd.read_excel(os.path.join(raw_fold, '용도별_차종별_시군구별_자동차1일평균주행거리(2021).xlsx'))
run.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1230 entries, 0 to 1229
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시도명     1230 non-null   object 
 1   시군구명    1230 non-null   object 
 2   차종      1230 non-null   object 
 3   전체      1230 non-null   float64
dtypes: float64(1), object(3)
memory usage: 38.6+ KB


In [27]:
run.tail()

,시도명,시군구명,차종,전체
1225,제주특별자치도,서귀포시,합계,35.10
1226,제주특별자치도,서귀포시,승용,36.10
1227,제주특별자치도,서귀포시,승합,37.20
1228,제주특별자치도,서귀포시,화물,32.50
1229,제주특별자치도,서귀포시,특수,38.40


In [28]:
len(set(df1['시군구명_수정'].unique()) - set(run['시군구명'].unique()))

0

In [29]:
run.tail()

,시도명,시군구명,차종,전체
1225,제주특별자치도,서귀포시,합계,35.10
1226,제주특별자치도,서귀포시,승용,36.10
1227,제주특별자치도,서귀포시,승합,37.20
1228,제주특별자치도,서귀포시,화물,32.50
1229,제주특별자치도,서귀포시,특수,38.40


In [30]:
df1.shape

(1149290, 34)

In [31]:
df2 = df1.merge(run, left_on=['시도명', '시군구명_수정', '차종'], right_on=['시도명', '시군구명', '차종'], how='left')
df2.shape

(1149290, 36)

In [32]:
df2.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명_x', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', 'Grade', '차령', '시군구명_수정',
       '시군구명_y', '전체'],
      dtype='object')

In [33]:
df2.drop('시군구명_y', axis=1, inplace=True)
df2.rename(columns={'시군구명_x':'시군구명', '전체':'1일평균주행거리'}, inplace=True)
df2.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', 'Grade', '차령', '시군구명_수정',
       '1일평균주행거리'],
      dtype='object')

In [34]:
df2[['시도명', '시군구명', '시군구명_수정', '차종', '1일평균주행거리']].tail()

,시도명,시군구명,시군구명_수정,차종,1일평균주행거리
1149285,충청남도,천안시 동남구,천안시,화물,52.90
1149286,전라북도,김제시,김제시,화물,42.60
1149287,전라북도,완주군,완주군,화물,57.00
1149288,경기도,시흥시,시흥시,승합,46.50
1149289,충청남도,태안군,태안군,화물,33.40


In [35]:
df2['VKT'] = df2['1일평균주행거리'] * 365

In [36]:
df2.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', 'Grade', '차령', '시군구명_수정',
       '1일평균주행거리', 'VKT'],
      dtype='object')

In [37]:
df2[['시도명', '시군구명', '차종', 'VKT']].tail()

,시도명,시군구명,차종,VKT
1149285,충청남도,천안시 동남구,화물,19308.50
1149286,전라북도,김제시,화물,15549.00
1149287,전라북도,완주군,화물,20805.00
1149288,경기도,시흥시,승합,16972.50
1149289,충청남도,태안군,화물,12191.00


In [38]:
df2['VKT'].isnull().sum()

0

### 배출가스 열화계수(DF) 계산
- 필요한 정보 : 연료, 차종, 차종유형, 연식, 차령

In [39]:
DF_col = ['연료', '차종', '차종유형', '연식', '차령']

In [40]:
df2[DF_col].head()

,연료,차종,차종유형,연식,차령
0,경유,승용,대형,2006,17
1,경유,승용,대형,2006,17
2,경유,승용,대형,2006,17
3,경유,승용,대형,2006,17
4,경유,승용,대형,2006,17


In [41]:
df2[DF_col].values[0]

array(['경유', '승용', '대형', 2006, 17], dtype=object)

In [42]:
# about 3.5s
DF_CO_list = []
DF_HC_list = []
DF_PM_list = []
DF_NOx_list = []
# '연료', '차종', '차종유형', '연식', '차령'
for fuel, car_type, car_size, car_birth, car_age in tqdm(df2[DF_col].values):    
    if fuel == '경유':
        if car_type == '승용':
            warranty = 5
            if car_birth <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
            DF_CO_list.append(DF_CO)
            DF_HC_list.append(DF_HC)
            DF_PM_list.append(DF_PM)
            DF_NOx_list.append(DF_NOx)
        elif car_type == '승합':
            if car_size == '소형' or car_size == '중형':
                warranty = 5
                if car_birth <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.05
                    DF_HC = 1 + (car_age - warranty)*0.05
                    DF_PM = 1 + (car_age - warranty)*0.05
                    DF_NOx = 1 + (car_age - warranty)*0.02
                    if DF_CO > 1.5:
                        DF_CO = 1.5
                    if DF_HC > 1.5:
                        DF_HC = 1.5
                    if DF_PM > 1.5:
                        DF_PM = 1.5
                    if DF_NOx > 1.2:
                        DF_NOx = 1.2
                DF_CO_list.append(DF_CO)
                DF_HC_list.append(DF_HC)
                DF_PM_list.append(DF_PM)
                DF_NOx_list.append(DF_NOx)
            elif car_size == '대형':
                warranty = 3
                if car_birth <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.05
                    DF_HC = 1 + (car_age - warranty)*0.05
                    DF_PM = 1 + (car_age - warranty)*0.05
                    DF_NOx = 1 + (car_age - warranty)*0.02
                    if DF_CO > 1.5:
                        DF_CO = 1.5
                    if DF_HC > 1.5:
                        DF_HC = 1.5
                    if DF_PM > 1.5:
                        DF_PM = 1.5
                    if DF_NOx > 1.2:
                        DF_NOx = 1.2
                DF_CO_list.append(DF_CO)
                DF_HC_list.append(DF_HC)
                DF_PM_list.append(DF_PM)
                DF_NOx_list.append(DF_NOx)
        elif car_type == '화물':
            # car_size : 경, 소, 중, 대
            warranty = 5
            if car_birth <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
            DF_CO_list.append(DF_CO)
            DF_HC_list.append(DF_HC)
            DF_PM_list.append(DF_PM)
            DF_NOx_list.append(DF_NOx)
        elif car_type == '특수':
            # car_size : 중, 대
            warranty = 5
            if car_birth <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
            DF_CO_list.append(DF_CO)
            DF_HC_list.append(DF_HC)
            DF_PM_list.append(DF_PM)
            DF_NOx_list.append(DF_NOx)
    else:
        pass

100%|██████████| 1149290/1149290 [00:03<00:00, 328462.93it/s]


In [43]:
df2.shape[0]

1149290

In [44]:
len(DF_CO_list), len(DF_HC_list), len(DF_PM_list), len(DF_NOx_list)

(1149290, 1149290, 1149290, 1149290)

In [45]:
df2['DF_CO'] = DF_CO_list
df2['DF_HC'] = DF_HC_list
df2['DF_PM'] = DF_PM_list
df2['DF_NOx'] = DF_NOx_list

In [46]:
check_DF_col = ['DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx']
df2.loc[df2['차령'] < 10, DF_col + check_DF_col].head()

,연료,차종,차종유형,연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx
1143229,경유,화물,중형,2018,5,1.00,1.00,1.00,1.00
1145884,경유,화물,대형,2018,5,1.00,1.00,1.00,1.00


### 저감장치 부착 효율(R) 계산
- DPF유무_수정 : 유 -> DPF로 가정하고 진행
- 휘발성 유기 화합물(Volatile Organic Compounds:VOC) : 생활주변에서 흔히 사용하는 탄화수소류가 거의 해당됨.
- 저감장치별 물질 제거 효율(%)

    장치종류|CO|VOC|PM
    :-:|:-:|:-:|:-:
    DPF|99.5|90|83.6
    pDPF|94.6|89.3|56
    DOC|85.4|72|35

In [47]:
df2['DPF유무_수정'].value_counts(dropna=False)

무       879463
유       251010
확인불가     18817
Name: DPF유무_수정, dtype: int64

In [48]:
df2['DPF유무_수정'].values[0]

'무'

In [49]:
R_CO_list = []
R_HC_list = []
R_PM_list = []
for dpf in tqdm(df2['DPF유무_수정'].values):
    if dpf == '유':
        R_CO = 99.5
        R_HC = 90
        R_PM = 83.6
    else:
        R_CO = 0
        R_HC = 0
        R_PM = 0
    R_CO_list.append(R_CO)
    R_HC_list.append(R_HC)
    R_PM_list.append(R_PM)

100%|██████████| 1149290/1149290 [00:00<00:00, 1322688.74it/s]


In [50]:
df2.shape[0]

1149290

In [51]:
len(R_CO_list), len(R_HC_list), len(R_PM_list)

(1149290, 1149290, 1149290)

In [52]:
df2['R_CO'] = R_CO_list
df2['R_HC'] = R_HC_list
df2['R_PM'] = R_PM_list

In [53]:
check_R_col = ['R_CO', 'R_HC', 'R_PM']
df2.loc[df2['DPF유무_수정'] == '유', DF_col + check_DF_col + check_R_col].head()

,연료,차종,차종유형,연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,R_HC,R_PM
3207,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,99.50,90,83.60
3208,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,99.50,90,83.60
3209,경유,승용,대형,2007,16,1.50,1.50,1.50,1.20,99.50,90,83.60
3210,경유,승용,대형,2007,16,1.50,1.50,1.50,1.20,99.50,90,83.60
3211,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,99.50,90,83.60


### 배출계수(EFi)


In [4]:
EFi_col = ['차종', '차종분류', '연료', '연식']
# KOSIS 차량속도(2017) 참고
V = 60.9
# 국가 대기오염물질 배출량 산정방법 편람(V)(2022) 부록 참고(for PM-2.5)
k = 0.92

In [ ]:
EFi_CO_list = []
EFi_HC_list = []
EFi_NOx_list = []
EFi_PM10_list = []
EFi_PM2_5_list = []
for car_type, car_size, fuel, car_birth in tqdm(df2[EFi_col].values):
    if (car_type == '승용') & (car_size == '경형'):
        if car_birth <= 2005:
            EFi_CO = 0.7392 * (V**-0.7524)
            EFi_HC = 0.0989 * (V**-0.6848)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.0839 * (V**-0.3420)
            EFi_PM2_5 = k * 0.0839 * (V**-0.3420)
        elif 2006 <= car_birth <= 2010:
            EFi_CO = 0.5775 * (V**-0.7524)
            EFi_HC = 0.0825 * (V**-0.6848)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.0420 * (V**-0.3420)
            EFi_PM2_5 = k * 0.0420 * (V**-0.3420)
        elif 2011 <= car_birth <= 2015:
            EFi_CO = 0.5141 * (V**-0.6792)
            EFi_HC = 0.3713 * (V**-0.7513)
            EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
            EFi_PM10 = 0.0225 * (V**-0.7264)
            EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
        elif 2016 <= car_birth:
            EFi_CO = 0.4574 * (V**-0.5215)
            EFi_HC = 0.1300 * (V**-0.7265)
            EFi_NOx = 2.7702 * (V**-0.3869)
            EFi_PM10 = 0.0225 * (V**-0.7264)
            EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
    elif (car_type == '승용') & (car_size == '소형'):
        if car_birth <= 2004:
            EFi_CO = 5.9672 * (V**-0.9534)
            EFi_HC = 0.6523 * (V**-1.0167)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.3861 * (V**-0.5093)
            EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
        elif car_birth == 2005:
            EFi_CO = 0.7392 * (V**-0.7524)
            EFi_HC = 0.0989 * (V**-0.6848)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.0839 * (V**-0.3420)
            EFi_PM2_5 = k * 0.0839 * (V**-0.3420)
        elif 2006 <= car_birth <= 2010:
            EFi_CO = 0.5775 * (V**-0.7524)
            EFi_HC = 0.0825 * (V**-0.6848)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.0420 * (V**-0.3420)
            EFi_PM2_5 = k * 0.0420 * (V**-0.3420)
        elif 2011 <= car_birth <= 2015:
            EFi_CO = 0.5141 * (V**-0.6792)
            EFi_HC = 0.3713 * (V**-0.7513)
            EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
            EFi_PM10 = 0.0225 * (V**-0.7264)
            EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
        elif 2016 <= car_birth:
            EFi_CO = 0.4574 * (V**-0.5215)
            EFi_HC = 0.1300 * (V**-0.7265)
            EFi_NOx = 2.7702 * (V**-0.3869)
            EFi_PM10 = 0.0225 * (V**-0.7264)
            EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
    elif (car_type == '승용') & (car_size == '중형'):
        if car_birth <= 2004:
            EFi_CO = 5.9672 * (V**-0.9534)
            EFi_HC = 0.6523 * (V**-1.0167)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.3861 * (V**-0.5093)
            EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
        elif car_birth == 2005:
            EFi_CO = 0.6930 * (V**-0.7524)
            EFi_HC = 0.1865 * (V**-0.6848)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.0723 * (V**-0.3420)
            EFi_PM2_5 = k * 0.0723 * (V**-0.3420)
        elif 2006 <= car_birth <= 2010:
            EFi_CO = 0.5414 * (V**-0.7524)
            EFi_HC = 0.0927 * (V**-0.6848)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.0396 * (V**-0.3420)
            EFi_PM2_5 = k * 0.0396 * (V**-0.3420)
        elif 2011 <= car_birth <= 2015:
            EFi_CO = 0.5141 * (V**-0.6792)
            EFi_HC = 0.3713 * (V**-0.7513)
            EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
            EFi_PM10 = 0.0225 * (V**-0.7264)
            EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
        elif 2016 <= car_birth:
            EFi_CO = 0.4574 * (V**-0.5215)
            EFi_HC = 0.1300 * (V**-0.7265)
            EFi_NOx = 2.7702 * (V**-0.3869)
            EFi_PM10 = 0.0225 * (V**-0.7264)
            EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
    elif (car_type == '승용') & (car_size == '대형'):
        if car_birth <= 2004:
            EFi_CO = 5.9672 * (V**-0.9534)
            EFi_HC = 0.6523 * (V**-1.0167)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.3861 * (V**-0.5093)
            EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
        elif car_birth == 2005:
            EFi_CO = 0.9609 * (V**-0.7524)
            EFi_HC = 0.1865 * (V**-0.6848)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.0723 * (V**-0.3420)
            EFi_PM2_5 = k * 0.0723 * (V**-0.3420)
        elif 2006 <= car_birth <= 2010:
            EFi_CO = 0.7507 * (V**-0.7524)
            EFi_HC = 0.1554 * (V**-0.6848)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.0361 * (V**-0.3420)
            EFi_PM2_5 = k * 0.0361 * (V**-0.3420)
        elif 2011 <= car_birth <= 2015:
            EFi_CO = 0.5141 * (V**-0.6792)
            EFi_HC = 0.3713 * (V**-0.7513)
            EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
            EFi_PM10 = 0.0225 * (V**-0.7264)
            EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
        elif 2016 <= car_birth:
            EFi_CO = 0.4574 * (V**-0.5215)
            EFi_HC = 0.1300 * (V**-0.7265)
            EFi_NOx = 2.7702 * (V**-0.3869)
            EFi_PM10 = 0.0225 * (V**-0.7264)
            EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
    elif (car_type == '승합') & (car_size == '소형'):
        if car_birth <= 1990:
            EFi_CO = 3.4539 * (V**-0.4266)
            EFi_HC = 0.9835 * (V**-0.5096)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 1.1412 * (V**-0.4324)
            EFi_PM2_5 = k * 1.1412 * (V**-0.4324)
        elif 1991 <= car_birth <= 1995:
            EFi_CO = 3.4539 * (V**-0.4266)
            EFi_HC = 1.6313 * (V**-0.7298)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.5999 * (V**-0.3294)
            EFi_PM2_5 = k * 0.5999 * (V**-0.3294)
        elif 1996 <= car_birth <= 1997:
            EFi_CO = 3.4539 * (V**-0.4266)
            EFi_HC = 1.1293 * (V**-0.6588)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.6408 * (V**-0.3596)
            EFi_PM2_5 = k * 0.6408 * (V**-0.3596)
        elif 1998 <= car_birth <= 1999:
            EFi_CO = 3.7564 * (V**-0.5175)
            EFi_HC = 1.1293 * (V**-0.6588)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.5168 * (V**-0.3596)
            EFi_PM2_5 = k * 0.5168 * (V**-0.3596)
        elif 2000 <= car_birth <= 2003:
            EFi_CO = 3.7564 * (V**-0.5175)
            EFi_HC = 1.1293 * (V**-0.6588)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.2894 * (V**-0.3596)
            EFi_PM2_5 = k * 0.2894 * (V**-0.3596)
        elif 2004 <= car_birth <= 2007:
            EFi_CO = 3.2797 * (V**-0.8887)
            EFi_HC = 0.1807 * (V**-0.6588)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.2067 * (V**-0.3596)
            EFi_PM2_5 = k * 0.2067 * (V**-0.3596)
        elif 2008 <= car_birth <= 2011:
            EFi_CO = 4.222 * (V**-1.4035)
            EFi_HC = 0.829 * (V**-1.0961)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.3111 * (V**-0.5125)
            EFi_PM2_5 = k * 0.3111 * (V**-0.5125)
        elif 2012 <= car_birth:
            EFi_CO = 4.222 * (V**-1.4035)
            EFi_HC = 0.829 * (V**-1.0961)
            EFi_NOx = 2.0217 * (V**-0.2645)
            EFi_PM10 = 0.1119 * (V**-0.5125)
            EFi_PM2_5 = k * 0.1119 * (V**-0.5125)
    elif (car_type == '승합') & (car_size == '중형'):
        if car_birth <= 1995:
            EFi_CO = 32.550 * (V**-0.4944)
            EFi_HC = 15.753 * (V**-0.5912)
            EFi_NOx = 40.692 * (V**-0.5590)
            EFi_PM10 = 5.4886 * (V**-0.5911)
            EFi_PM2_5 = k * 5.4886 * (V**-0.5911)
        elif 1996 <= car_birth <= 1997:
            EFi_CO = 16.410 * (V**-0.3790)
            EFi_HC = 4.2324 * (V**-0.3926)
            EFi_NOx = 22.804 * (V**-0.4660)
            EFi_PM10 = 1.6593 * (V**-0.3935)
            EFi_PM2_5 = k * 1.6593 * (V**-0.3935)
        elif 1998 <= car_birth <= 2000:
            EFi_CO = 16.410 * (V**-0.3790)
            EFi_HC = 4.2324 * (V**-0.3926)
            EFi_NOx = 25.708 * (V**-0.4772)
            EFi_PM10 = 1.6593 * (V**-0.3935)
            EFi_PM2_5 = k * 1.6593 * (V**-0.3935)
        elif 2001 <= car_birth <= 2004:
            EFi_CO = 16.378 * (V**-0.5340)
            EFi_HC = 5.8477 * (V**-0.5466)
            EFi_NOx = 25.436 * (V**-0.4656)
            EFi_PM10 = 1.2848 * (V**-0.4715)
            EFi_PM2_5 = k * 1.2848 * (V**-0.4715)
        elif 2005 <= car_birth <= 2007:
            EFi_CO = 15.256 * (V**-0.7448)
            EFi_HC = 2.0502 * (V**-0.6504)
            EFi_NOx = 15.001 * (V**-0.4528)
            EFi_PM10 = 0.2979 * (V**-0.4008)
            EFi_PM2_5 = k * 1.0457 * (V**-0.4527)
        elif 2008 <= car_birth <= 2010:
            EFi_CO = 8.1771 * (V**-0.7725)
            EFi_HC = 1.2991 * (V**-0.6538)
            EFi_NOx = 17.3032 * (V**-0.3660)
            EFi_PM10 = 0.0539 * (V**-0.5182)
            EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
        elif 2011 <= car_birth <= 2014:
            EFi_CO = 4.5201 * (V**-0.7279)
            EFi_HC = 1.6826 * (V**-0.8045)
            EFi_NOx = 17.2485 * (V**-0.4040)
            EFi_PM10 = 0.0469 * (V**-0.4674)
            EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
        elif 2015 <= car_birth:
            EFi_CO = 7.4065 * (V**-0.5995)
            EFi_HC = 2.4562 * (V**-1.3145)
            EFi_NOx = 42.7393 * (V**-1.2949)
            EFi_PM10 = 0.0081 * (V**-0.5182)
            EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
    elif (car_type == '승합') & (car_size == '대형'):
        if car_birth <= 1995:
            EFi_CO = 28.205 * (V**-0.5337)
            EFi_HC = 6.1146 * (V**-0.4979)
            EFi_NOx = 41.346 * (V**-0.3645)
            EFi_PM10 = 5.2158 * (V**-0.5048)
            EFi_PM2_5 = k * 5.2158 * (V**-0.5048)
        elif 1996 <= car_birth <= 1997:
            EFi_CO = 23.205 * (V**-0.5425)
            EFi_HC = 6.5657 * (V**-0.5431)
            EFi_NOx = 42.1379 * (V**-0.3786)
            EFi_PM10 = 2.4911 * (V**-0.4149)
            EFi_PM2_5 = k * 2.4911 * (V**-0.4149)
        elif 1998 <= car_birth <= 2000:
            EFi_CO = 23.205 * (V**-0.5425)
            EFi_HC = 6.5657 * (V**-0.5431)
            EFi_NOx = 42.1379 * (V**-0.3786)
            EFi_PM10 = 1.4432 * (V**-0.3870)
            EFi_PM2_5 = k * 1.4432 * (V**-0.3870)
        elif car_birth == 2001:
            EFi_CO = 21.348 * (V**-0.5806)
            EFi_HC = 6.6390 * (V**-0.5760)
            EFi_NOx = 36.7191 * (V**-0.3548)
            EFi_PM10 = 0.9375 * (V**-0.3910)
            EFi_PM2_5 = k * 0.9375 * (V**-0.3910)
        elif 2002 <= car_birth <= 2004:
            EFi_CO = 21.348 * (V**-0.5806)
            EFi_HC = 6.6390 * (V**-0.5760)
            EFi_NOx = 36.7191 * (V**-0.3548)
            EFi_PM10 = 1.1507 * (V**-0.4804)
            EFi_PM2_5 = k * 1.1507 * (V**-0.4804)
        elif 2005 <= car_birth <= 2007:
            EFi_CO = 9.6452 * (V**-0.5291)
            EFi_HC = 3.2339 * (V**-0.7436)
            EFi_NOx = 30.5870 * (V**-0.3548)
            EFi_PM10 = 0.4657 * (V**-0.5634)
            EFi_PM2_5 = k * 0.4657 * (V**-0.5634)
        elif 2008 <= car_birth <= 2010:
            EFi_CO = 6.8493 * (V**-0.6506)
            EFi_HC = 1.7177 * (V**-0.6781)
            EFi_NOx = 40.7564 * (V**-0.4757)
            EFi_PM10 = 0.2418 * (V**-0.4727)
            EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
        elif 2011 <= car_birth <= 2014:
            EFi_CO = 5.4607 * (V**-0.2990)
            EFi_HC = 0.8863 * (V**-0.6933)
            EFi_NOx = 40.3729 * (V**-0.5386)
            EFi_PM10 = 0.2125 * (V**-0.4650)
            EFi_PM2_5 = k * 0.2125 * (V**-0.4650)
        elif 2015 <= car_birth:
            EFi_CO = 11.4415 * (V**-0.8036)
            EFi_HC = 0.6774 * (V**-0.8321)
            EFi_NOx = 112.1229 * (V**-1.6393)
            EFi_PM10 = 0.0363 * (V**-0.4727)
            EFi_PM2_5 = k * 0.0363 * (V**-0.4727)
    elif (car_type == '화물') & (car_size == '소형'):
        if car_birth <= 1990:
            EFi_CO = 4.5854 * (V**-0.3613)
            EFi_HC = 0.4840 * (V**-0.2756)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.8117 * (V**-0.4071)
            EFi_PM2_5 = k * 0.8117 * (V**-0.4071)
        elif 1991 <= car_birth <= 1995:
            EFi_CO = 3.4774 * (V**-0.3483)
            EFi_HC = 0.4844 * (V**-0.3288)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.6188 * (V**-0.4540)
            EFi_PM2_5 = k * 0.6188 * (V**-0.4540)
        elif 1996 <= car_birth <= 1997:
            EFi_CO = 3.3934 * (V**-0.3837)
            EFi_HC = 0.4955 * (V**-0.3393)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.7037 * (V**-0.5357)
            EFi_PM2_5 = k * 0.7037 * (V**-0.5357)
        elif 1998 <= car_birth <= 1999:
            EFi_CO = 4.0896 * (V**-0.6083)
            EFi_HC = 0.6122 * (V**-0.5684)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.6157 * (V**-0.5357)
            EFi_PM2_5 = k * 0.6157 * (V**-0.5357)
        elif 2000 <= car_birth <= 2003:
            EFi_CO = 4.0896 * (V**-0.6083)
            EFi_HC = 0.6122 * (V**-0.5684)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.4838 * (V**-0.5357)
            EFi_PM2_5 = k * 0.4838 * (V**-0.5357)
        elif 2004 <= car_birth <= 2007:
            EFi_CO = 3.2797 * (V**-0.8887)
            EFi_HC = 0.1807 * (V**-0.6588)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.2067 * (V**-0.3596)
            EFi_PM2_5 = k * 0.2067 * (V**-0.3596)
        elif 2008 <= car_birth <= 2011:
            EFi_CO = 4.222 * (V**-1.4035)
            EFi_HC = 0.829 * (V**-1.0961)
            EFi_NOx = 24.3491 * (V**-0.7277)
            EFi_PM10 = 0.3111 * (V**-0.5125)
            EFi_PM2_5 = k * 0.3111 * (V**-0.5125)
        elif 2012 <= car_birth <= 2016:
            EFi_CO = 4.222 * (V**-1.4035)
            EFi_HC = 0.829 * (V**-1.0961)
            EFi_NOx = 2.0217 * (V**-0.2645)
            EFi_PM10 = 0.1119 * (V**-0.5125)
            EFi_PM2_5 = k * 0.1119 * (V**-0.5125)
# 수정중...
# 화물 중형
# 화물 대형

In [101]:
# # about 20.5s
# today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
# df1.iloc[:500000].to_csv(os.path.join(raw_fold, f'4등급 차량 리스트 양식 result 최종 Grade 추가({today_date})01.csv'), index=False, encoding='cp949')
# df1.iloc[500000:].to_csv(os.path.join(raw_fold, f'4등급 차량 리스트 양식 result 최종 Grade 추가({today_date})02.csv'), index=False, encoding='cp949')

# 코드 마지막